In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import time
import numpy as np

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
#Using Pandas to create dataframe from an excel file
df = pd.read_excel("Compiled_Nashville_September_Property_Sales_GetCoordsFinal.xlsx")
df['Sale Price'] = df['Sale Price'].apply(lambda x: "${:.1f}k".format((x/1000)))
df.head()

,Parcel ID,Land Use,Property Address,Property City,Sale Date,Sale Price,Legal Reference,Sold As Vacant,Multiple Parcels Involved in Sale,Zone,Coordinates
0,053 05 0A 008.00,SINGLE FAMILY,1100 CHEYENNE CIR,MADISON,2021-09-07,$304.0k,20210921-0126315,No,No,6,"36.2524142, -86.6706970"
1,006 00 0 010.00,SINGLE FAMILY,3450 GREER RD,GOODLETTSVILLE,2021-09-17,$405.0k,20210920-0125863,No,No,5,"36.3835361, -86.7832459"
2,004 00 0 075.00,SINGLE FAMILY,8565 WHITES CREEK PIKE,JOELTON,2021-09-22,$666.5k,20211001-0131984,No,No,5,"36.3811685, -86.8945705"
3,004 00 0 193.00,SINGLE FAMILY,8560 COOPERTOWN RD,JOELTON,2021-09-20,$620.0k,20210924-0128377,No,No,5,"36.3808100, -86.9061216"
4,004 00 0 139.00,SINGLE FAMILY,8409 RILEY ADCOCK RD,JOELTON,2021-09-17,$227.5k,20210923-0127923,No,No,5,"36.3729140, -86.8836440"


In [3]:
#Getting the Latitude and Longitude from the Coordinates Column
df[["Latitude","Longitude"]] = df["Coordinates"].str.split(",", expand = True)
df.head()

,Parcel ID,Land Use,Property Address,Property City,Sale Date,Sale Price,Legal Reference,Sold As Vacant,Multiple Parcels Involved in Sale,Zone,Coordinates,Latitude,Longitude
0,053 05 0A 008.00,SINGLE FAMILY,1100 CHEYENNE CIR,MADISON,2021-09-07,$304.0k,20210921-0126315,No,No,6,"36.2524142, -86.6706970",36.2524142,-86.6706970
1,006 00 0 010.00,SINGLE FAMILY,3450 GREER RD,GOODLETTSVILLE,2021-09-17,$405.0k,20210920-0125863,No,No,5,"36.3835361, -86.7832459",36.3835361,-86.7832459
2,004 00 0 075.00,SINGLE FAMILY,8565 WHITES CREEK PIKE,JOELTON,2021-09-22,$666.5k,20211001-0131984,No,No,5,"36.3811685, -86.8945705",36.3811685,-86.8945705
3,004 00 0 193.00,SINGLE FAMILY,8560 COOPERTOWN RD,JOELTON,2021-09-20,$620.0k,20210924-0128377,No,No,5,"36.3808100, -86.9061216",36.3808100,-86.9061216
4,004 00 0 139.00,SINGLE FAMILY,8409 RILEY ADCOCK RD,JOELTON,2021-09-17,$227.5k,20210923-0127923,No,No,5,"36.3729140, -86.8836440",36.3729140,-86.8836440


In [4]:
#Converting Lat and Long to a float
df['Latitude'] = pd.to_numeric(df['Latitude'],errors='coerce')
df['Longitude'] = pd.to_numeric(df['Longitude'],errors='coerce')
df.dtypes

Parcel ID                                    object
Land Use                                     object
Property Address                             object
Property City                                object
Sale Date                            datetime64[ns]
Sale Price                                   object
Legal Reference                              object
Sold As Vacant                               object
Multiple Parcels Involved in Sale            object
Zone                                          int64
Coordinates                                  object
Latitude                                    float64
Longitude                                   float64
dtype: object

In [42]:
# Create a new Data Frame that contains information that is needed.
fast_food_df = df[["Property Address", "Property City", "Sale Price", "Latitude", "Longitude"]].copy()
fast_food_df["Restaurants"] = ""
fast_food_df.head(10)
#Adding the following: 
# restaurant 
# supermarket 
# liquor_store 
# school, church 
# convenience_store 
# drugstore 
# supermarket 
# hospital 
# bar

,Property Address,Property City,Sale Price,Latitude,Longitude,Restaurants
0,1100 CHEYENNE CIR,MADISON,$304.0k,36.252414,-86.670697,
1,3450 GREER RD,GOODLETTSVILLE,$405.0k,36.383536,-86.783246,
2,8565 WHITES CREEK PIKE,JOELTON,$666.5k,36.381169,-86.894571,
3,8560 COOPERTOWN RD,JOELTON,$620.0k,36.380810,-86.906122,
4,8409 RILEY ADCOCK RD,JOELTON,$227.5k,36.372914,-86.883644,
5,7840 WHITES CREEK PIKE,JOELTON,$575.0k,36.355360,-86.878055,
6,3560 BAXTER RD,JOELTON,$805.0k,36.352787,-86.885008,
7,7564 BIDWELL RD,JOELTON,$313.5k,36.343657,-86.893232,
8,208 A SWIFT DR,GOODLETTSVILLE,$333.0k,36.335490,-86.720321,
9,109 SWIFT DR,GOODLETTSVILLE,$350.0k,36.334522,-86.719428,


In [45]:
#Setting up the gmaps search and nearby proximity
params  = {
    "radius": 1600,
    "type": "restaurant",
    "key": g_key
    }

fast_five_df = fast_food_df.head(5)
fast_five_df

for index, row in fast_five_df.iterrows():
#Getting Latitude and Longitude
    lat = row["Latitude"]
    lng = row["Longitude"]
    params["location"] = f"{lat},{lng}"

#Getting base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    time.sleep(3)

    food_places = requests.get(base_url, params=params).json()

    try:        
        fast_five_df.loc[index, "Restaurants"] = len(food_places["results"])
    except:
        print("No fast food found... skipping")




# for index, row in fast_food_df.iterrows():
# #Getting Latitude and Longitude
#     lat = row["Latitude"]
#     lng = row["Longitude"]
#     params["location"] = f"{lat},{lng}"

# #Getting base url
#     base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#     time.sleep(3)

#     food_places = requests.get(base_url, params=params).json()

#     try:
#         fast_food_df.loc[index, "Fast Food"] = food_places["results"][0]["name"]
#     except (IndexError):
#         print("No fast food found... skipping")


C:\Users\Alecia\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [46]:
fast_five_df

,Property Address,Property City,Sale Price,Latitude,Longitude,Restaurants
0,1100 CHEYENNE CIR,MADISON,$304.0k,36.252414,-86.670697,11
1,3450 GREER RD,GOODLETTSVILLE,$405.0k,36.383536,-86.783246,0
2,8565 WHITES CREEK PIKE,JOELTON,$666.5k,36.381169,-86.894571,1
3,8560 COOPERTOWN RD,JOELTON,$620.0k,36.380810,-86.906122,1
4,8409 RILEY ADCOCK RD,JOELTON,$227.5k,36.372914,-86.883644,1


In [8]:
fast_food_df = fast_food_df.dropna()
fast_food_df.head()

,Property Address,Property City,Sale Price,Latitude,Longitude,Fast Food
0,1100 CHEYENNE CIR,MADISON,$304.0k,36.252414,-86.670697,
1,3450 GREER RD,GOODLETTSVILLE,$405.0k,36.383536,-86.783246,
2,8565 WHITES CREEK PIKE,JOELTON,$666.5k,36.381169,-86.894571,
3,8560 COOPERTOWN RD,JOELTON,$620.0k,36.380810,-86.906122,
4,8409 RILEY ADCOCK RD,JOELTON,$227.5k,36.372914,-86.883644,


In [40]:
info_box_template = """
<dl>
<dt>Nearest Resturant</dt><dd>{Fast Food}</dd>
<dt>City</dt><dd>{Property City}</dd>
<dt>Propety Address</dt><dd>{Property Address}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
food_info = [info_box_template.format(**row) for index, row in fast_food_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = fast_food_df[["Latitude", "Longitude"]]

In [41]:
marker_layer = gmaps.marker_layer(locations)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))